In [1]:
import math
import os
import time
import pickle as pkl
import tensorflow as tf
import pandas as pd
import numpy as np
import numpy.linalg as la

from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass

In [3]:
def load_data(path):
    data = pd.read_csv(path)
    data = data.drop(columns = ['Unnamed: 0'])
    data = data[1:]
    print(data.shape, (data == 0).sum().sum())
    return data

In [4]:
def preprocess_data(data, time_len, rate, seq_len, pre_len):
    train_size = int(time_len * rate)
    train_data = data[0:train_size]
    test_data = data[train_size:time_len]
    
    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(train_data) - seq_len - pre_len):
        a = train_data[i: i + seq_len + pre_len]
        trainX.append(a[0 : seq_len])
        trainY.append(a[seq_len : seq_len + pre_len])
    for i in range(len(test_data) - seq_len -pre_len):
        b = test_data[i: i + seq_len + pre_len]
        testX.append(b[0 : seq_len])
        testY.append(b[seq_len : seq_len + pre_len])
        
    trainX = np.array(trainX)
    trainY = np.array(trainY)
    testX = np.array(testX)
    testY = np.array(testY)
    return trainX, trainY, testX, testY

In [5]:
def generate_model(seq_len, node_num, pre_len):
    model = tf.keras.Sequential([
        tf.keras.layers.GRU(units=64, return_sequences=True, input_shape = [seq_len, node_num]),
        tf.keras.layers.GRU(units=64),
        tf.keras.layers.Dense(pre_len * node_num)
    ])
    
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    return model

In [6]:
def train_model(model, train_datas, realY):
    trainX, trainY, testX, testY = train_datas[0], train_datas[1], train_datas[2], train_datas[3]
    print(trainX.shape, trainY.shape, testX.shape, testY.shape, realY.shape)
    
    model.fit(trainX, trainY, epochs=30, batch_size=32)
    y_pred = model.predict(testX)
    print(y_pred.shape)
    
    rmse = math.sqrt(mean_squared_error(realY, y_pred))
    mae = mean_absolute_error(realY, y_pred)
    mape = mean_absolute_percentage_error(realY, y_pred)
    f_norm = la.norm(realY - y_pred) / la.norm(realY)
    acc = 1-f_norm
    
    print(rmse, mae, mape, acc)

In [7]:
path_0 = '../Data/gangnam/speed_gangnam_0.csv'
path_5 = '../Data/gangnam/speed_gangnam_5.csv'
path_10 = '../Data/gangnam/speed_gangnam_10.csv'
path_20 = '../Data/gangnam/speed_gangnam_20.csv'

In [8]:
train_rate = 0.8
seq_len = 12

In [9]:
mainData = load_data(path_0)
data_0 = load_data(path_0)
data_5 = load_data(path_5)
data_10 = load_data(path_10)
data_20 = load_data(path_20)

(2879, 506) 0
(2879, 506) 0
(2879, 506) 71975
(2879, 506) 143950
(2879, 506) 290779


# Pred = 15min

In [10]:
pre_len = 3

In [11]:
time_len = mainData.shape[0]
node_num = mainData.shape[1]

print(time_len, node_num)

_, _, _, realY = preprocess_data(mainData, time_len, train_rate, seq_len, pre_len)
realY = realY.reshape([-1, node_num * pre_len])
realY.shape

2879 506


(561, 1518)

### Unobserved nodes = 0%

In [12]:
time_len = data_0.shape[0]
node_num = data_0.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_0, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(2288, 12, 506) (2288, 1518) (561, 12, 506) (561, 1518)
(2288, 12, 506) (2288, 1518) (561, 12, 506) (561, 1518) (561, 1518)
Epoch 1/30
72/72 [==============================] - 5s 10ms/step - loss: 806.7126 - mae: 26.0258
Epoch 2/30
72/72 [==============================] - 1s 10ms/step - loss: 580.1199 - mae: 21.3167
Epoch 3/30
72/72 [==============================] - 1s 10ms/step - loss: 421.4374 - mae: 17.4281
Epoch 4/30
72/72 [==============================] - 1s 10ms/step - loss: 308.9812 - mae: 14.3124
Epoch 5/30
72/72 [==============================] - 1s 10ms/step - loss: 230.4549 - mae: 11.8993
Epoch 6/30
72/72 [==============================] - 1s 10ms/step - loss: 176.3775 - mae: 10.1159
Epoch 7/30
72/72 [==============================] - 1s 10ms/step - loss: 139.5918 - mae: 8.8259
Epoch 8/30
72/72 [==============================] - 1s 10ms/step - loss: 114.6700 - mae: 7.9184
Epoch 9/30
72/72 [==============================] - 1s 10ms/step - loss: 97.7525 - mae: 7.2917
Epoch 1

### Unobserved nodes = 5%

In [13]:
time_len = data_5.shape[0]
node_num = data_5.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_5, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(2288, 12, 506) (2288, 1518) (561, 12, 506) (561, 1518)
(2288, 12, 506) (2288, 1518) (561, 12, 506) (561, 1518) (561, 1518)
Epoch 1/30
72/72 [==============================] - 3s 10ms/step - loss: 773.8951 - mae: 24.8342
Epoch 2/30
72/72 [==============================] - 1s 10ms/step - loss: 558.0941 - mae: 20.3597
Epoch 3/30
72/72 [==============================] - 1s 10ms/step - loss: 406.4309 - mae: 16.6717
Epoch 4/30
72/72 [==============================] - 1s 10ms/step - loss: 298.7034 - mae: 13.7042
Epoch 5/30
72/72 [==============================] - 1s 10ms/step - loss: 223.2189 - mae: 11.4071
Epoch 6/30
72/72 [==============================] - 1s 10ms/step - loss: 171.0629 - mae: 9.7033
Epoch 7/30
72/72 [==============================] - 1s 10ms/step - loss: 135.4461 - mae: 8.4710
Epoch 8/30
72/72 [==============================] - 1s 10ms/step - loss: 111.1809 - mae: 7.5976
Epoch 9/30
72/72 [==============================] - 1s 10ms/step - loss: 94.6781 - mae: 6.9934
Epoch 10

### Unobserved nodes = 10%

In [14]:
time_len = data_10.shape[0]
node_num = data_10.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_10, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(2288, 12, 506) (2288, 1518) (561, 12, 506) (561, 1518)
(2288, 12, 506) (2288, 1518) (561, 12, 506) (561, 1518) (561, 1518)
Epoch 1/30
72/72 [==============================] - 3s 10ms/step - loss: 717.8806 - mae: 23.3397
Epoch 2/30
72/72 [==============================] - 1s 10ms/step - loss: 516.0658 - mae: 19.1064
Epoch 3/30
72/72 [==============================] - 1s 10ms/step - loss: 374.2145 - mae: 15.6165
Epoch 4/30
72/72 [==============================] - 1s 11ms/step - loss: 273.8015 - mae: 12.8099
Epoch 5/30
72/72 [==============================] - 1s 11ms/step - loss: 203.8889 - mae: 10.6348
Epoch 6/30
72/72 [==============================] - 1s 10ms/step - loss: 155.9523 - mae: 9.0294
Epoch 7/30
72/72 [==============================] - 1s 10ms/step - loss: 123.5039 - mae: 7.8738
Epoch 8/30
72/72 [==============================] - 1s 10ms/step - loss: 101.6237 - mae: 7.0666
Epoch 9/30
72/72 [==============================] - 1s 10ms/step - loss: 86.8039 - mae: 6.5073
Epoch 10

### Unobserved nodes = 20%

In [15]:
time_len = data_20.shape[0]
node_num = data_20.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_20, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(2288, 12, 506) (2288, 1518) (561, 12, 506) (561, 1518)
(2288, 12, 506) (2288, 1518) (561, 12, 506) (561, 1518) (561, 1518)
Epoch 1/30
72/72 [==============================] - 5s 12ms/step - loss: 646.6235 - mae: 20.7738
Epoch 2/30
72/72 [==============================] - 1s 9ms/step - loss: 468.8259 - mae: 17.0642
Epoch 3/30
72/72 [==============================] - 1s 8ms/step - loss: 342.5020 - mae: 13.9890
Epoch 4/30
72/72 [==============================] - 1s 14ms/step - loss: 252.5701 - mae: 11.5089
Epoch 5/30
72/72 [==============================] - 1s 10ms/step - loss: 189.3965 - mae: 9.6051
Epoch 6/30
72/72 [==============================] - 1s 9ms/step - loss: 145.6743 - mae: 8.1885
Epoch 7/30
72/72 [==============================] - 1s 10ms/step - loss: 115.8011 - mae: 7.1675
Epoch 8/30
72/72 [==============================] - 1s 8ms/step - loss: 95.3876 - mae: 6.4436
Epoch 9/30
72/72 [==============================] - 1s 8ms/step - loss: 81.4051 - mae: 5.9372
Epoch 10/30
72/

# Pred = 30min

In [16]:
pre_len = 6

In [17]:
time_len = mainData.shape[0]
node_num = mainData.shape[1]

print(time_len, node_num)

_, _, _, realY = preprocess_data(mainData, time_len, train_rate, seq_len, pre_len)
realY = realY.reshape([-1, node_num * pre_len])
realY.shape

2879 506


(558, 3036)

### Unobserved nodes = 0%

In [18]:
time_len = data_0.shape[0]
node_num = data_0.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_0, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(2285, 12, 506) (2285, 3036) (558, 12, 506) (558, 3036)
(2285, 12, 506) (2285, 3036) (558, 12, 506) (558, 3036) (558, 3036)
Epoch 1/30
72/72 [==============================] - 4s 10ms/step - loss: 808.6727 - mae: 26.0610
Epoch 2/30
72/72 [==============================] - 1s 11ms/step - loss: 582.4365 - mae: 21.3561
Epoch 3/30
72/72 [==============================] - 1s 10ms/step - loss: 423.2843 - mae: 17.4898
Epoch 4/30
72/72 [==============================] - 1s 10ms/step - loss: 310.4212 - mae: 14.3473
Epoch 5/30
72/72 [==============================] - 1s 10ms/step - loss: 231.6079 - mae: 11.9352
Epoch 6/30
72/72 [==============================] - 1s 11ms/step - loss: 177.2372 - mae: 10.1431
Epoch 7/30
72/72 [==============================] - 1s 11ms/step - loss: 140.2319 - mae: 8.8500
Epoch 8/30
72/72 [==============================] - 1s 10ms/step - loss: 115.1081 - mae: 7.9372
Epoch 9/30
72/72 [==============================] - 1s 10ms/step - loss: 98.0524 - mae: 7.3023
Epoch 1

### Unobserved nodes = 5%

In [19]:
time_len = data_5.shape[0]
node_num = data_5.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_5, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(2285, 12, 506) (2285, 3036) (558, 12, 506) (558, 3036)
(2285, 12, 506) (2285, 3036) (558, 12, 506) (558, 3036) (558, 3036)
Epoch 1/30
72/72 [==============================] - 3s 10ms/step - loss: 775.4438 - mae: 24.8588
Epoch 2/30
72/72 [==============================] - 1s 10ms/step - loss: 561.5782 - mae: 20.4393
Epoch 3/30
72/72 [==============================] - 1s 10ms/step - loss: 409.7519 - mae: 16.7560
Epoch 4/30
72/72 [==============================] - 1s 10ms/step - loss: 301.5279 - mae: 13.7883
Epoch 5/30
72/72 [==============================] - 1s 10ms/step - loss: 225.3685 - mae: 11.4803
Epoch 6/30
72/72 [==============================] - 1s 10ms/step - loss: 172.6487 - mae: 9.7573
Epoch 7/30
72/72 [==============================] - 1s 10ms/step - loss: 136.5837 - mae: 8.5125
Epoch 8/30
72/72 [==============================] - 1s 10ms/step - loss: 112.0284 - mae: 7.6304
Epoch 9/30
72/72 [==============================] - 1s 10ms/step - loss: 95.2162 - mae: 7.0126
Epoch 10

### Unobserved nodes = 10%

In [20]:
time_len = data_10.shape[0]
node_num = data_10.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_10, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(2285, 12, 506) (2285, 3036) (558, 12, 506) (558, 3036)
(2285, 12, 506) (2285, 3036) (558, 12, 506) (558, 3036) (558, 3036)
Epoch 1/30
72/72 [==============================] - 4s 10ms/step - loss: 719.2311 - mae: 23.3605
Epoch 2/30
72/72 [==============================] - 1s 10ms/step - loss: 515.0559 - mae: 19.0867
Epoch 3/30
72/72 [==============================] - 1s 10ms/step - loss: 372.6842 - mae: 15.5718
Epoch 4/30
72/72 [==============================] - 1s 10ms/step - loss: 272.5636 - mae: 12.7587
Epoch 5/30
72/72 [==============================] - 1s 10ms/step - loss: 202.8777 - mae: 10.6016
Epoch 6/30
72/72 [==============================] - 1s 10ms/step - loss: 155.1681 - mae: 8.9996
Epoch 7/30
72/72 [==============================] - 1s 10ms/step - loss: 122.8546 - mae: 7.8493
Epoch 8/30
72/72 [==============================] - 1s 10ms/step - loss: 101.1887 - mae: 7.0472
Epoch 9/30
72/72 [==============================] - 1s 10ms/step - loss: 86.5006 - mae: 6.4945
Epoch 10

### Unobserved nodes = 20%

In [21]:
time_len = data_20.shape[0]
node_num = data_20.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_20, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(2285, 12, 506) (2285, 3036) (558, 12, 506) (558, 3036)
(2285, 12, 506) (2285, 3036) (558, 12, 506) (558, 3036) (558, 3036)
Epoch 1/30
72/72 [==============================] - 3s 7ms/step - loss: 647.2098 - mae: 20.7811
Epoch 2/30
72/72 [==============================] - 1s 7ms/step - loss: 465.8816 - mae: 16.9959
Epoch 3/30
72/72 [==============================] - 1s 7ms/step - loss: 339.0916 - mae: 13.8921
Epoch 4/30
72/72 [==============================] - 1s 7ms/step - loss: 249.4648 - mae: 11.4156
Epoch 5/30
72/72 [==============================] - 1s 7ms/step - loss: 186.9843 - mae: 9.5222
Epoch 6/30
72/72 [==============================] - 1s 7ms/step - loss: 143.8737 - mae: 8.1199
Epoch 7/30
72/72 [==============================] - 1s 7ms/step - loss: 114.4531 - mae: 7.1152
Epoch 8/30
72/72 [==============================] - 1s 7ms/step - loss: 94.4379 - mae: 6.4046
Epoch 9/30
72/72 [==============================] - 1s 7ms/step - loss: 80.7181 - mae: 5.9104
Epoch 10/30
72/72 [

# Pred = 45min

In [22]:
pre_len = 9

In [23]:
time_len = mainData.shape[0]
node_num = mainData.shape[1]

print(time_len, node_num)

_, _, _, realY = preprocess_data(mainData, time_len, train_rate, seq_len, pre_len)
realY = realY.reshape([-1, node_num * pre_len])
realY.shape

2879 506


(555, 4554)

### Unobserved nodes = 0%

In [24]:
time_len = data_0.shape[0]
node_num = data_0.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_0, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(2282, 12, 506) (2282, 4554) (555, 12, 506) (555, 4554)
(2282, 12, 506) (2282, 4554) (555, 12, 506) (555, 4554) (555, 4554)
Epoch 1/30
72/72 [==============================] - 3s 10ms/step - loss: 806.3347 - mae: 26.0248
Epoch 2/30
72/72 [==============================] - 1s 10ms/step - loss: 583.8218 - mae: 21.3985
Epoch 3/30
72/72 [==============================] - 1s 10ms/step - loss: 425.4803 - mae: 17.5392
Epoch 4/30
72/72 [==============================] - 1s 10ms/step - loss: 312.5775 - mae: 14.4169
Epoch 5/30
72/72 [==============================] - 1s 10ms/step - loss: 233.4969 - mae: 12.0011
Epoch 6/30
72/72 [==============================] - 1s 10ms/step - loss: 178.7457 - mae: 10.1983
Epoch 7/30
72/72 [==============================] - 1s 10ms/step - loss: 141.3374 - mae: 8.8924
Epoch 8/30
72/72 [==============================] - 1s 10ms/step - loss: 115.9137 - mae: 7.9676
Epoch 9/30
72/72 [==============================] - 1s 10ms/step - loss: 98.6348 - mae: 7.3253
Epoch 1

### Unobserved nodes = 5%

In [25]:
time_len = data_5.shape[0]
node_num = data_5.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_5, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(2282, 12, 506) (2282, 4554) (555, 12, 506) (555, 4554)
(2282, 12, 506) (2282, 4554) (555, 12, 506) (555, 4554) (555, 4554)
Epoch 1/30
72/72 [==============================] - 3s 10ms/step - loss: 775.9582 - mae: 24.8671
Epoch 2/30
72/72 [==============================] - 1s 10ms/step - loss: 562.3333 - mae: 20.4599
Epoch 3/30
72/72 [==============================] - 1s 10ms/step - loss: 410.2799 - mae: 16.7754
Epoch 4/30
72/72 [==============================] - 1s 10ms/step - loss: 301.9554 - mae: 13.8034
Epoch 5/30
72/72 [==============================] - 1s 10ms/step - loss: 225.8057 - mae: 11.4951
Epoch 6/30
72/72 [==============================] - 1s 10ms/step - loss: 173.1512 - mae: 9.7738
Epoch 7/30
72/72 [==============================] - 1s 10ms/step - loss: 137.0257 - mae: 8.5311
Epoch 8/30
72/72 [==============================] - 1s 10ms/step - loss: 112.3048 - mae: 7.6438
Epoch 9/30
72/72 [==============================] - 1s 10ms/step - loss: 95.3962 - mae: 7.0207
Epoch 10

### Unobserved nodes = 10%

In [26]:
time_len = data_10.shape[0]
node_num = data_10.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_10, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(2282, 12, 506) (2282, 4554) (555, 12, 506) (555, 4554)
(2282, 12, 506) (2282, 4554) (555, 12, 506) (555, 4554) (555, 4554)
Epoch 1/30
72/72 [==============================] - 3s 8ms/step - loss: 717.1906 - mae: 23.3240
Epoch 2/30
72/72 [==============================] - 1s 7ms/step - loss: 516.6307 - mae: 19.1209
Epoch 3/30
72/72 [==============================] - 1s 7ms/step - loss: 375.1440 - mae: 15.6372
Epoch 4/30
72/72 [==============================] - 1s 7ms/step - loss: 274.7296 - mae: 12.8357
Epoch 5/30
72/72 [==============================] - 1s 8ms/step - loss: 204.6306 - mae: 10.6574
Epoch 6/30
72/72 [==============================] - 1s 8ms/step - loss: 156.6098 - mae: 9.0521
Epoch 7/30
72/72 [==============================] - 1s 8ms/step - loss: 123.8804 - mae: 7.8919
Epoch 8/30
72/72 [==============================] - 1s 8ms/step - loss: 101.8549 - mae: 7.0746
Epoch 9/30
72/72 [==============================] - 1s 8ms/step - loss: 86.9398 - mae: 6.5131
Epoch 10/30
72/72

### Unobserved nodes = 20%

In [27]:
time_len = data_20.shape[0]
node_num = data_20.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_20, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(2282, 12, 506) (2282, 4554) (555, 12, 506) (555, 4554)
(2282, 12, 506) (2282, 4554) (555, 12, 506) (555, 4554) (555, 4554)
Epoch 1/30
72/72 [==============================] - 3s 10ms/step - loss: 647.4947 - mae: 20.7910
Epoch 2/30
72/72 [==============================] - 1s 10ms/step - loss: 468.2525 - mae: 17.0475
Epoch 3/30
72/72 [==============================] - 1s 10ms/step - loss: 341.5739 - mae: 13.9598
Epoch 4/30
72/72 [==============================] - 1s 10ms/step - loss: 251.6000 - mae: 11.4789
Epoch 5/30
72/72 [==============================] - 1s 10ms/step - loss: 188.6505 - mae: 9.5767
Epoch 6/30
72/72 [==============================] - 1s 10ms/step - loss: 145.1847 - mae: 8.1671
Epoch 7/30
72/72 [==============================] - 1s 11ms/step - loss: 115.4133 - mae: 7.1516
Epoch 8/30
72/72 [==============================] - 1s 10ms/step - loss: 95.1217 - mae: 6.4334
Epoch 9/30
72/72 [==============================] - 1s 10ms/step - loss: 81.2136 - mae: 5.9293
Epoch 10/3

# Pred = 60min

In [28]:
pre_len = 12

In [29]:
time_len = mainData.shape[0]
node_num = mainData.shape[1]

print(time_len, node_num)

_, _, _, realY = preprocess_data(mainData, time_len, train_rate, seq_len, pre_len)
realY = realY.reshape([-1, node_num * pre_len])
realY.shape

2879 506


(552, 6072)

### Unobserved nodes = 0%

In [30]:
time_len = data_0.shape[0]
node_num = data_0.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_0, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(2279, 12, 506) (2279, 6072) (552, 12, 506) (552, 6072)
(2279, 12, 506) (2279, 6072) (552, 12, 506) (552, 6072) (552, 6072)
Epoch 1/30
72/72 [==============================] - 3s 10ms/step - loss: 808.1274 - mae: 26.0650
Epoch 2/30
72/72 [==============================] - 1s 10ms/step - loss: 582.7302 - mae: 21.3763
Epoch 3/30
72/72 [==============================] - 1s 10ms/step - loss: 423.6467 - mae: 17.4927
Epoch 4/30
72/72 [==============================] - 1s 10ms/step - loss: 310.8903 - mae: 14.3622
Epoch 5/30
72/72 [==============================] - 1s 10ms/step - loss: 231.8007 - mae: 11.9471
Epoch 6/30
72/72 [==============================] - 1s 10ms/step - loss: 177.3833 - mae: 10.1485
Epoch 7/30
72/72 [==============================] - 1s 10ms/step - loss: 140.3777 - mae: 8.8539
Epoch 8/30
72/72 [==============================] - 1s 10ms/step - loss: 115.2203 - mae: 7.9394
Epoch 9/30
72/72 [==============================] - 1s 10ms/step - loss: 98.1188 - mae: 7.3048
Epoch 1

### Unobserved nodes = 5%

In [31]:
time_len = data_5.shape[0]
node_num = data_5.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_5, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(2279, 12, 506) (2279, 6072) (552, 12, 506) (552, 6072)
(2279, 12, 506) (2279, 6072) (552, 12, 506) (552, 6072) (552, 6072)
Epoch 1/30
72/72 [==============================] - 3s 10ms/step - loss: 774.5044 - mae: 24.8497
Epoch 2/30
72/72 [==============================] - 1s 10ms/step - loss: 560.0652 - mae: 20.4054
Epoch 3/30
72/72 [==============================] - 1s 10ms/step - loss: 408.4252 - mae: 16.7248
Epoch 4/30
72/72 [==============================] - 1s 10ms/step - loss: 300.3773 - mae: 13.7515
Epoch 5/30
72/72 [==============================] - 1s 10ms/step - loss: 224.5333 - mae: 11.4560
Epoch 6/30
72/72 [==============================] - 1s 11ms/step - loss: 172.0806 - mae: 9.7366
Epoch 7/30
72/72 [==============================] - 1s 10ms/step - loss: 136.2943 - mae: 8.5006
Epoch 8/30
72/72 [==============================] - 1s 10ms/step - loss: 111.8395 - mae: 7.6239
Epoch 9/30
72/72 [==============================] - 1s 10ms/step - loss: 95.1613 - mae: 7.0129
Epoch 10

### Unobserved nodes = 10%

In [32]:
time_len = data_10.shape[0]
node_num = data_10.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_10, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(2279, 12, 506) (2279, 6072) (552, 12, 506) (552, 6072)
(2279, 12, 506) (2279, 6072) (552, 12, 506) (552, 6072) (552, 6072)
Epoch 1/30
72/72 [==============================] - 3s 10ms/step - loss: 717.4296 - mae: 23.3303
Epoch 2/30
72/72 [==============================] - 1s 10ms/step - loss: 516.4919 - mae: 19.1197
Epoch 3/30
72/72 [==============================] - 1s 10ms/step - loss: 374.8029 - mae: 15.6368
Epoch 4/30
72/72 [==============================] - 1s 10ms/step - loss: 274.3617 - mae: 12.8157
Epoch 5/30
72/72 [==============================] - 1s 11ms/step - loss: 204.3982 - mae: 10.6541
Epoch 6/30
72/72 [==============================] - 1s 10ms/step - loss: 156.3458 - mae: 9.0423
Epoch 7/30
72/72 [==============================] - 1s 10ms/step - loss: 123.7812 - mae: 7.8865
Epoch 8/30
72/72 [==============================] - 1s 11ms/step - loss: 101.7947 - mae: 7.0730
Epoch 9/30
72/72 [==============================] - 1s 10ms/step - loss: 86.8908 - mae: 6.5118
Epoch 10

### Unobserved nodes = 20%

In [33]:
time_len = data_20.shape[0]
node_num = data_20.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_20, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(2279, 12, 506) (2279, 6072) (552, 12, 506) (552, 6072)
(2279, 12, 506) (2279, 6072) (552, 12, 506) (552, 6072) (552, 6072)
Epoch 1/30
72/72 [==============================] - 3s 10ms/step - loss: 646.4757 - mae: 20.7790
Epoch 2/30
72/72 [==============================] - 1s 10ms/step - loss: 467.5652 - mae: 17.0356
Epoch 3/30
72/72 [==============================] - 1s 10ms/step - loss: 341.3817 - mae: 13.9512
Epoch 4/30
72/72 [==============================] - 1s 10ms/step - loss: 251.6111 - mae: 11.4777
Epoch 5/30
72/72 [==============================] - 1s 10ms/step - loss: 188.7725 - mae: 9.5778
Epoch 6/30
72/72 [==============================] - 1s 10ms/step - loss: 145.2337 - mae: 8.1713
Epoch 7/30
72/72 [==============================] - 1s 10ms/step - loss: 115.4766 - mae: 7.1556
Epoch 8/30
72/72 [==============================] - 1s 11ms/step - loss: 95.1825 - mae: 6.4351
Epoch 9/30
72/72 [==============================] - 1s 11ms/step - loss: 81.3407 - mae: 5.9349
Epoch 10/3